# Alternating Least Squares (ALS) and Maximum Likelihood Estimation (MLE)

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict, deque

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

## Load data

In [2]:
ml_ratings_df = pd.read_csv(os.path.join("data", "movielens_small", "ratings.csv")).rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
ml_movies_df = pd.read_csv(os.path.join("data", "movielens_small", "movies.csv")).rename(columns={'movieId': 'item_id'})
ml_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')

# Filter the data to reduce the number of movies
seed = 6789
rng = np.random.RandomState(seed=seed)
left_ids = rng.choice(ml_movies_df['item_id'], size=90, replace=False)
left_ids = list(set(left_ids).union(set([1, 318, 1193, 1208, 1214, 1721, 2959, 3578, 4306, 109487])))

ml_ratings_df = ml_ratings_df.loc[ml_ratings_df['item_id'].isin(left_ids)]
ml_movies_df = ml_movies_df.loc[ml_movies_df['item_id'].isin(left_ids)]
ml_df = ml_df.loc[ml_df['item_id'].isin(left_ids)]

display(ml_movies_df.head(10))

print("Number of interactions left: {}".format(len(ml_ratings_df)))

,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
143,171,Jeffrey (1995),Comedy|Drama
194,228,Destiny Turns on the Radio (1995),Comedy
199,233,Exotica (1994),Drama
230,267,Major Payne (1995),Comedy
277,318,"Shawshank Redemption, The (1994)",Crime|Drama
313,355,"Flintstones, The (1994)",Children|Comedy|Fantasy
379,435,Coneheads (1993),Comedy|Sci-Fi
419,481,Kalifornia (1993),Drama|Thriller
615,780,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller


Number of interactions left: 2761


## Shift item ids and user ids so that they are consecutive

In [3]:
interactions_df = ml_ratings_df.copy()

unique_item_ids = interactions_df['item_id'].unique()
item_id_mapping = dict(zip(unique_item_ids, list(range(len(unique_item_ids)))))
item_id_reverse_mapping = dict(zip(list(range(len(unique_item_ids))), unique_item_ids))
unique_user_ids = interactions_df['user_id'].unique()
user_id_mapping = dict(zip(unique_user_ids, list(range(len(unique_user_ids)))))
user_id_reverse_mapping = dict(zip(list(range(len(unique_user_ids))), unique_user_ids))

interactions_df['item_id'] = interactions_df['item_id'].map(item_id_mapping)
interactions_df['user_id'] = interactions_df['user_id'].map(user_id_mapping)

display(interactions_df.head(10))

,user_id,item_id,rating,timestamp
0,0,0,4.0,964982703
42,0,1,3.0,964984086
72,0,2,4.0,964983250
75,0,3,4.0,964981855
97,0,4,4.0,964980985
192,0,5,5.0,964983282
216,0,6,4.0,964981725
219,0,7,5.0,964980668
232,1,8,3.0,1445714835
235,1,7,4.0,1445714885


## Get the number of items and users

In [4]:
n_items = np.max(interactions_df['item_id']) + 1
n_users = np.max(interactions_df['user_id']) + 1

print("n_items={}\nn_users={}".format(n_items, n_users))

n_items=100
n_users=555


## Get the user-item interaction matrix

In [5]:
# mapping to int is necessary because of how iterrows works
r = np.zeros(shape=(n_users, n_items))
for idx, interaction in interactions_df.iterrows():
    r[int(interaction['user_id'])][int(interaction['item_id'])] = 1
    
print(r)

[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]


## Least squares

**Task 1.** Write a method perform_least_squares for performing the matrix calculation of the linear model coefficients minimizing the square loss for a given 2D dataset (where the first column is x - the explanatory variable, the second column is y - the target variable). The method should return a tuple of fitted theta_0 and theta_1. To fit the bias term (theta_0) you have to concatenate 1 to every input vector, e.g. $[0.2]$ -> $[1, 0.2]$.

The interface of the method should be as follows:
    
    perform_least_squares(data)
    
Compare your solution to the optimal one found by sklearn.linear_model.LinearRegression.
    
Plot the data (as scatterplot) and the fit (as lineplot) on a single seaborn chart.

In [ ]:
data = np.array([
    [0.2, 0.52],
    [0.5, 1.36],
    [0.9, 1.00],
    [1.3, 1.69],
    [1.5, 2.58],
    [2.8, 2.34]
])


def perform_least_squares(data):
    x, y = data[:, 0], data[:, 1]
    
    ########################
    # Write your code here #
    ########################
    
    return theta_0, theta_1


theta_0, theta_1 = perform_least_squares(data)

min_x = min(data[:, 0]) * 0.9
max_x = max(data[:, 0]) * 1.1
x = np.linspace(min_x, max_x, 100)
y = theta_1 * x + theta_0

from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(data[:, 0].reshape(-1, 1), data[:, 1].flatten())
y_lin = reg.predict(x.reshape(-1, 1))

fig = plt.figure(tight_layout=True)
fig.set_size_inches(16, 9)
ax1 = fig.add_subplot(1, 1, 1)

sns.scatterplot(x=data[:, 0], y=data[:, 1], ax=ax1, label="data")
sns.lineplot(x=x, y=y, ax=ax1, label="your fit")
sns.lineplot(x=x, y=y_lin, ax=ax1, label="sklearn fit")

plt.show()

## Alternating Least Squares (ALS)

Generate negative interactions - copy your code from notebook 8.

In [ ]:
n_neg_per_pos = 5
interactions_pos_neg_df = interactions_df[['user_id', 'item_id']].copy()

# Indicate positive interactions

interactions_pos_neg_df['interacted'] = 1

# Generate negative interactions

negative_interactions = []

########################
# Write your code here #
########################

**Task 2.** From interactions_pos_neg_df create two dictionaries:
- interactions_pos_neg_user_dict where keys are user ids and values are lists of item ids the user has interacted with,
- interactions_pos_neg_item_dict where keys are item ids and values are lists of user ids who interacted with the item.

In [ ]:
########################
# Write your code here #
########################



print("Interactions of user_id=0")
print(interactions_pos_neg_user_dict[0])
display(interactions_pos_neg_df.loc[interactions_pos_neg_df['user_id'] == 0])
print()
print("Interactions of user_id=1")
print(interactions_pos_neg_user_dict[1])
display(interactions_pos_neg_df.loc[interactions_pos_neg_df['user_id'] == 1])
print()
print("Interactions of item_id=0")
print(interactions_pos_neg_item_dict[0])
display(interactions_pos_neg_df.loc[interactions_pos_neg_df['item_id'] == 0])
print()
print("Interactions of item_id=1")
print(interactions_pos_neg_item_dict[1])
display(interactions_pos_neg_df.loc[interactions_pos_neg_df['item_id'] == 1])

Initialize user and item embeddings of size embedding_dim - use your code from notebook 8.

In [ ]:
embedding_dim = 2
rng = np.random.RandomState(seed=seed)

########################
# Write your code here #
########################



print(user_repr_matrix)
print()
print(item_repr_matrix)

**Task 3.** Write the perform_mf_als_step method which takes item/user representations interacting_reprs for items the user has interacted with or for users who interacted with a given item (a 2D matrix where each row corresponds to an item representation/embedding), interaction values r_ui which is a 1D matrix of binary values to be predicted (the value from the interaction matrix - interactions_pos_neg_df), regularization constant reg_l, and performs a single step of Alternating Least Squares for matrix factorization as described in Koren, Bell, Volinksy "Matrix Factorization Techniques for Recommender Systems". You can use sklearn.linear_model.Ridge for that (x should be interacting_reprs, y should be r_ui and the result will be the new representation for the given user/item, you also have to disable fitting the intercept/bias in the model by setting fit_intercept=False). The method should return the new representation.

In [ ]:
def perform_mf_als_step(interacting_reprs, r_ui, reg_l):
    ########################
    # Write your code here #
    ########################
        
    
    return new_repr


# Test

interacting_item_repr_matrix = np.array([[0.75, 0.15], [-0.12, 0.32], [0.44, -0.95]])
r_ui = np.array([1, 0, 0])
reg_l = 0.1
new_user_repr = perform_mf_als_step(interacting_item_repr_matrix, r_ui, reg_l)
print(new_user_repr)

**Task 4.** Write the perform_mf_als_epoch method which takes interactions_df, user_repr_matrix, item_repr_matrix, regularization constant reg_l as input, and then:
- iterate over all users of interactions_df, perform perform_mf_als_step for every user (use the interactions_pos_neg_user_dict dictionary to get representations (rows in item_repr_matrix) and interactions ("interacted" column from interactions_df) for the items the the user has interacted with), update the appropriate user representation in the user_repr_matrix,
- iterate over all items of interactions_df, perform perform_mf_als_step for every item (use the interactions_pos_neg_item_dict dictionary to get representations (rows in user_repr_matrix) and interactions ("interacted" column from interactions_df) for the users who interacted with the given item), update the appropriate item representation in the item_repr_matrix,
- return updated user_repr_matrix, item_repr_matrix.

To obtain consistent results run the cell creating base user and item representations again before running this cell.

In [ ]:
def perform_mf_als_epoch(interactions_df, user_repr_matrix, item_repr_matrix, reg_l):
    ########################
    # Write your code here #
    ########################
    
    
    return user_repr_matrix, item_repr_matrix


# Test

user_repr_matrix, item_repr_matrix \
    = perform_mf_als_epoch(interactions_pos_neg_df, user_repr_matrix, item_repr_matrix, reg_l)

print(user_repr_matrix)
print()
print(item_repr_matrix)

**Task 5.** Write the perform_mf_als_training method which takes interactions_df, user_repr_matrix, item_repr_matrix, regularization constant reg_l, epsilon as input and performs the following steps until the loss change between epochs is smaller than epsilon for three consecutive epochs:
- perform perform_mf_als_epoch, 
- calculate the loss for the user and item representations returned by perform_mf_als_epoch using the formula from the Koren, Bell, Volinksy "Matrix Factorization Techniques for Recommender Systems" paper.
Finally return a tuple containing user_repr_matrix, item_repr_matrix and the final loss.

To obtain consistent results run the cell creating base user and item representations and task 4 cell again before running this cell.

In [ ]:
from livelossplot import PlotLosses

def perform_mf_als_training(interactions_df, user_repr_matrix, item_repr_matrix, reg_l, epsilon):
    liveloss = PlotLosses()
    
    epoch = 0
    
    ########################
    # Write your code here #
    ########################
        

        # Save and print epoch losses (this should be at the end of the loop over epochs)

        training_last_avg_loss = total_loss / len(interactions_df)

        if epoch >= 3: # A bound on epoch prevents showing extremely high losses in the first epochs
            logs = {'loss': training_last_avg_loss}
            liveloss.update(logs)
            liveloss.send()
        epoch += 1

    return user_repr_matrix, item_repr_matrix, training_last_avg_loss

# Test

epsilon = 0.01
user_repr_matrix, item_repr_matrix, training_last_avg_loss \
    = perform_mf_als_training(interactions_pos_neg_df, user_repr_matrix, item_repr_matrix, reg_l, epsilon)

print(user_repr_matrix)
print()
print(item_repr_matrix)
print()
print(training_last_avg_loss)

### Plot movie representations

Remember that they don't have to be good as only two dimensions have been used. But still try to find if you can assign any meaning to those dimensions based on your knowledge about plotted movies. You can open the image in new tab and enlarge.

In [ ]:
fig = plt.figure(tight_layout=True)
fig.set_size_inches(64, 36)
ax1 = fig.add_subplot(1, 1, 1)

sns.scatterplot(x=item_repr_matrix[:, 0], y=item_repr_matrix[:, 1], ax=ax1, color='red')
sns.scatterplot(x=user_repr_matrix[:, 0], y=user_repr_matrix[:, 1], ax=ax1, color='blue')

for i in range(len(item_repr_matrix)):
    title = ml_movies_df.loc[ml_movies_df['item_id'] == item_id_reverse_mapping[i], 'title'].iloc[0]
    plt.text(x=item_repr_matrix[i, 0] + 1 / 500, y=item_repr_matrix[i, 1] + 1 / 500, s=title, 
             fontdict=dict(color='red', size=8))
    
for i in range(len(user_repr_matrix)):
    plt.text(x=user_repr_matrix[i, 0] + 1 / 500, y=user_repr_matrix[i, 1] + 1 / 500, s=i, 
             fontdict=dict(color='blue', size=8))

plt.show()

## Maximum Likelihood Estimation for binary data

Generate the true logistic model and random data from it.

In [ ]:
def log_curve(x, theta_0, theta_1):
    return 1 / (1 + np.exp(-theta_0 - theta_1 * x))

real_theta_0 = -10
real_theta_1 = 1.5

data_x = rng.uniform(0, 10, size=20)
p = log_curve(data_x, real_theta_0, real_theta_1)
data_y = rng.binomial(1, p)

min_x = min(data_x) * 0.9
max_x = max(data_x) * 1.1
x_grid = np.linspace(min_x, max_x, 100)
y_log_curve = log_curve(x_grid, real_theta_0, real_theta_1)

fig = plt.figure(tight_layout=True)
fig.set_size_inches(16, 9)
ax1 = fig.add_subplot(1, 1, 1)

sns.scatterplot(x=data_x, y=data_y, ax=ax1, label="data")
sns.lineplot(x=x_grid, y=y_log_curve, ax=ax1, label="true logistic curve")

plt.show()

**Task 6.** Code a loss function which returns the negative likelihood of observing a given dataset (data_x, data_y) given that the true model is a logistic curve with parameters theta_0, theta_1:
$$P(y = 1) = \frac{1}{1 + e^{-\theta_0 - \theta_1 * x}}$$

In [ ]:
def loss_mle(params):
    theta_0, theta_1 = params['theta_0'], params['theta_1']
    ########################
    # Write your code here #
    ########################
    
    return -likelihood

Use hyperopt to minimize the loss defined in the previous task on data_x, data_y. Plot the data, the true model and the MLE fit on one seaborn chart. How does it compare to the MSE fit in the next cell? Which one looks better? Experiment with different datasets.

In [ ]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

space = {"theta_0": hp.uniform("theta_0", -50, 50),
         "theta_1": hp.loguniform("theta_1", np.log(0.01), np.log(100.0))}

succeded = False
n_tries = 3
t = 0
best_mle_param_set = {'theta_0': None, 'theta_1': None}
while not succeded and t < n_tries:
    try:
        trials = Trials()
        best_mle_param_set = fmin(loss_mle, space=space, algo=tpe.suggest, max_evals=100, show_progressbar=True, trials=trials)
        succeded = True
    except:
        t += 1
        traceback.print_exc()

# Best params

print("theta_0 = {}".format(best_mle_param_set['theta_0']))
print("theta_1 = {}".format(best_mle_param_set['theta_1']))

In [ ]:
y_log_curve_mle_fit = log_curve(x_grid, best_mle_param_set['theta_0'], best_mle_param_set['theta_1'])

fig = plt.figure(tight_layout=True)
fig.set_size_inches(16, 9)
ax1 = fig.add_subplot(1, 1, 1)

sns.scatterplot(x=data_x, y=data_y, ax=ax1, label="data")
sns.lineplot(x=x_grid, y=y_log_curve, ax=ax1, label="true logistic curve")
sns.lineplot(x=x_grid, y=y_log_curve_mle_fit, ax=ax1, label="mle fit logistic curve")

plt.show()

In [ ]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def loss_mse(params):
    theta_0, theta_1 = params['theta_0'], params['theta_1']
    mse = np.sum(np.power(data_y - log_curve(data_x, theta_0, theta_1), 2))
    return mse

space = {"theta_0": hp.uniform("theta_0", -50, 50),
         "theta_1": hp.loguniform("theta_1", np.log(0.01), np.log(100.0))}

succeded = False
n_tries = 3
t = 0
best_mse_param_set = {'theta_0': None, 'theta_1': None}
while not succeded and t < n_tries:
    try:
        trials = Trials()
        best_mse_param_set = fmin(loss_mse, space=space, algo=tpe.suggest, max_evals=100, show_progressbar=True, trials=trials)
        succeded = True
    except:
        t += 1
        traceback.print_exc()

# Best params

print("theta_0 = {}".format(best_mse_param_set['theta_0']))
print("theta_1 = {}".format(best_mse_param_set['theta_1']))

In [ ]:
y_log_curve_mse_fit = log_curve(x_grid, best_mse_param_set['theta_0'], best_mse_param_set['theta_1'])

fig = plt.figure(tight_layout=True)
fig.set_size_inches(16, 9)
ax1 = fig.add_subplot(1, 1, 1)

sns.scatterplot(x=data_x, y=data_y, ax=ax1, label="data")
sns.lineplot(x=x_grid, y=y_log_curve, ax=ax1, label="true logistic curve")
sns.lineplot(x=x_grid, y=y_log_curve_mse_fit, ax=ax1, label="mse fit logistic curve")

plt.show()